This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '-cj6xKSCgAEAAzBsoTzV'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import json
import requests
import pandas
import math

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params = {
    "api_key": "-cj6xKSCgAEAAzBsoTzV",
    "start_date": "2020-01-01",
    "end_date": "2020-01-01"
}
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X", params=params)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
if response.status_code == 200:
    data = json.loads(response.content)

In [5]:
print(json.dumps(data, indent=2))

{
  "dataset": {
    "id": 10095370,
    "dataset_code": "AFX_X",
    "database_code": "FSE",
    "name": "Carl Zeiss Meditec (AFX_X)",
    "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
    "refreshed_at": "2020-12-01T14:48:09.907Z",
    "newest_available_date": "2020-12-01",
    "oldest_available_date": "2000-06-07",
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "frequency": "daily",
    "type": "Time Series",
    "premium": false,
    "limit": null,
    "transform": null,
    "column_index": null,
    "start_date": "2020-01-01",
    "end_date": "2020-01-01",
    "data": [],
    "collapse": null,
    "order": null,
    "database_id": 6129
  }
}


In [6]:
print(data["dataset"]["column_names"])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [7]:
params = {
    "api_key": "-cj6xKSCgAEAAzBsoTzV",
    "start_date": "2017-01-01",
    "end_date": "2017-12-31"
}
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X", params=params)

2. Convert the returned JSON object into a Python dictionary.

In [8]:
if response.status_code == 200:
    data = json.loads(response.content)

In [9]:
data = data["dataset"]["data"]

In [10]:
data[0:5]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None],
 ['2017-12-27',
  51.45,
  51.89,
  50.76,
  51.82,
  None,
  57452.0,
  2957018.0,
  None,
  None,
  None],
 ['2017-12-22',
  51.05,
  51.5,
  50.92,
  51.32,
  None,
  71165.0,
  3641949.0,
  None,
  None,
  None],
 ['2017-12-21',
  51.16,
  51.52,
  50.9,
  51.4,
  None,
  120649.0,
  6179433.0,
  None,
  None,
  None]]

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [11]:
high = data[0][1]
low = data[0][1]
for d in data:
    if d[1] is not None and d[1] > high:
        high = d[1]
    if d[1] is not None and d[1] < low:
        low = d[1]
print("Max Openning => {}".format(high))
print("Min Openning => {}".format(low))

Max Openning => 53.11
Min Openning => 34.0


4. What was the largest change in any one day (based on High and Low price)?

In [12]:
max_change = float(data[0][2]) - float(data[0][3])
for d in data:
    if d[2] is not None and d[3] is not None:
        change = d[2] - d[3]
        if change > max_change:
            max_change = change;
    
print("Max Change on high and low => {}".format(max_change))

Max Change on high and low => 2.8100000000000023


5. What was the largest change between any two days (based on Closing Price)?

In [13]:
min_close = data[0][4]
max_close = data[0][4]

In [14]:
for d in data:
    if d[4] is not None:
        if d[4] < min_close:
            min_close = d[4]
        if d[4] > max_close:
            max_close = d[4]
print("Max change on high and low close => ", max_close - min_close)

Max change on high and low close =>  19.03


6. What was the average daily trading volume during this year?

In [15]:
sum = 0
for d in data:
    sum = sum + d[6]

In [16]:
print("Average daily volume =>", sum/len(data))

Average daily volume => 89124.33725490196


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [17]:
volums = []

In [18]:
for d in data:
    volums.append(d[6])

In [19]:
len(volums)

255

In [20]:
volums

[34640.0,
 40660.0,
 57452.0,
 71165.0,
 120649.0,
 50587.0,
 137313.0,
 129733.0,
 204080.0,
 132981.0,
 128434.0,
 87911.0,
 71817.0,
 109157.0,
 48123.0,
 88730.0,
 83023.0,
 94385.0,
 101733.0,
 123019.0,
 67342.0,
 42669.0,
 102180.0,
 50350.0,
 38834.0,
 91142.0,
 78502.0,
 97252.0,
 54107.0,
 89373.0,
 67593.0,
 67672.0,
 76286.0,
 90245.0,
 120268.0,
 94195.0,
 48152.0,
 60716.0,
 56911.0,
 37958.0,
 56319.0,
 56245.0,
 74472.0,
 56319.0,
 47730.0,
 43042.0,
 43375.0,
 55707.0,
 59991.0,
 45263.0,
 65837.0,
 49246.0,
 43362.0,
 49180.0,
 69455.0,
 65860.0,
 44059.0,
 66760.0,
 94804.0,
 115706.0,
 95313.0,
 99821.0,
 157234.0,
 76600.0,
 51321.0,
 56224.0,
 79955.0,
 105194.0,
 57838.0,
 65546.0,
 44037.0,
 107144.0,
 65157.0,
 68801.0,
 52828.0,
 103273.0,
 59881.0,
 64320.0,
 71006.0,
 66351.0,
 105288.0,
 151474.0,
 157888.0,
 97804.0,
 98156.0,
 47130.0,
 69734.0,
 65213.0,
 70269.0,
 95376.0,
 68812.0,
 72886.0,
 131361.0,
 104676.0,
 64334.0,
 127682.0,
 183412.0,
 175161

In [21]:
def find_median(arr):
    newArr = sorted(arr)
    mod = len(newArr) % 2
    if mod == 0:
        index =  len(newArr) / 2
        first_value = newArr[index]
        second_value = newArr[index + 1]
        return (first_value + second_value) / 2
    else:
        index =  math.floor(len(newArr) / 2)
        return newArr[index + 1]

In [22]:
find_median(volums)

76600.0

In [23]:
sorted(volums)[128]

76600.0